<a href="https://colab.research.google.com/github/yleessam/2501_dl_env_colab/blob/main/1_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84%EB%B0%A9%EB%B2%95_%EB%B6%93%EA%BD%83%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

25.1.12 로컬 테스트완료

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>div.container { width:100% !important; }</style>"))

In [ ]:
#!pip install -U scikit-learn


# 인공신경망 딥러닝 모델 구현

In [26]:
import tensorflow
tensorflow.__version__

'2.17.1'

## iris 데이터 불러오기

In [19]:
from sklearn import datasets

In [20]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=1)

## 싸이킷런을 이용한 구현

In [22]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000)

In [23]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [24]:
mlp.score(X_test, y_test)

1.0

In [25]:
pred = mlp.predict(X_test)

In [9]:
import pandas as pd
pd.crosstab(pred, y_test) #교차분류표
# pd.crosstab(np.argmax(pred, axis=1), np.argmax(y_test, axis=1)) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


## 텐서플로우를 이용한 구현

In [27]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [28]:
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])

In [29]:
W1 = tf.Variable(tf.random_normal([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(x, W1)+b1)

In [30]:
W2 = tf.Variable(tf.random_normal([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

In [31]:
W3 = tf.Variable(tf.random_normal([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [32]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [33]:
import pandas as pd

feed_train = {x: X_train, y: pd.get_dummies(y_train).values}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss))

Step 0, 3.19758
Step 100, 0.66335
Step 200, 0.32563
Step 300, 0.19987
Step 400, 0.13184
Step 500, 0.09941
Step 600, 0.08212
Step 700, 0.07192
Step 800, 0.06444
Step 900, 0.06065


In [35]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {x: X_test, y: pd.get_dummies(y_test).values}
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [18]:
sess.close()

## 케라스를 이용한 구현
* 여기부터는 커널(또는 세션)을 재시작한 후 실행하세요.

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X, y = iris.data, iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [2]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [3]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(shape=(4,))) #입력
model.add(Dense(50, activation='sigmoid')) #은닉1
model.add(Dense(30, activation='sigmoid')) #은닉2
model.add(Dense(3, activation='softmax')) #출력

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(X_train, y_train, epochs=300, verbose=1)

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3517 - loss: 1.6827
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3299 - loss: 1.5610 
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3611 - loss: 1.3702 
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3309 - loss: 1.3333 
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3371 - loss: 1.2419 
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3882 - loss: 1.1344 
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3267 - loss: 1.1451 
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3101 - loss: 1.1247
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3111 - loss: 1.1001  
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3465 - loss: 1.0758 
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3285 - loss: 1.0718 
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6479 - los

In [6]:
model.evaluate(X_test, y_test) # 출력값은 loss와 accuracy

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0636 


[0.06198687478899956, 1.0]

## 파이토치를 이용한 구현

In [ ]:
# # CPU 버전 설치 (최신 버전)
# ! pip install torch

# # GPU 지원 버전 설치 (최신 버전)
# ! pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [7]:
import torch
print(torch.__version__)

2.5.1+cu121


In [8]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [9]:
import torch
import torch.nn as nn

# 모델 클래스 정의
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 3)
        self.softmax = nn.Softmax(dim=1)
        # 가중치 초기화 (Xavier 초기화)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# 모델 생성
model = CustomModel()

In [10]:
# 모델 정의를 아래처럼 Sequential을 이용할 수 있음
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

In [11]:
# 손실 함수 및 옵티마이저 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 모델 학습
for epoch in range(1000):
    inputs = torch.from_numpy(X_train).float()
    labels = torch.from_numpy(y_train).long()

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

In [12]:
# 모델 평가
with torch.no_grad():
    inputs = torch.from_numpy(X_test).float()
    labels = torch.from_numpy(y_test).long()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == labels).sum().item() / len(y_test)

print(f"Test Loss:{loss.item()}, Test Accuracy:{accuracy}")

Test Loss:0.5716530680656433, Test Accuracy:1.0


### 파이토치의 Sequential 이용

In [13]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [14]:
# 넘파이 배열을 토치 텐서로 변환
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 모델 정의
model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

# 가중치 초깃값 지정
for layer in model:
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [16]:
# 모델 학습
for epoch in range(300):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 모델 평가
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print("Loss: {}, Accuracy: {}".format(loss.item(), accuracy))

Loss: 0.6322924494743347, Accuracy: 0.9777777777777777
